In [1]:
import pandas as pd 
import os 
PATH_TRAIN = os.path.join("..","data","train_data_ext","train.csv")
PATH_TEST = os.path.join("..","data","train_data_ext","test.csv")
PATH_VAL = os.path.join("..","data","train_data_ext","val.csv")

In [2]:
train_data = pd.read_csv(PATH_TRAIN)
test_data = pd.read_csv(PATH_TEST)
val_data = pd.read_csv(PATH_VAL)

In [3]:
train_data.head()

,text,label
0,"I didn't feel humiliated, which was a surprise...",sadness
1,"I didn't feel humiliated, which was a surprise...",sadness
2,I can go from feeling so hopeless to so damned...,sadness
3,I'm grabbing a minute to post because I feel i...,anger
4,I am ever feeling nostalgic about the fireplac...,love


In [4]:
train_data = train_data.drop_duplicates()

In [5]:
train_data.head()

,text,label
0,"I didn't feel humiliated, which was a surprise...",sadness
2,I can go from feeling so hopeless to so damned...,sadness
3,I'm grabbing a minute to post because I feel i...,anger
4,I am ever feeling nostalgic about the fireplac...,love
5,"I am feeling extremely grouchy today, and to b...",anger


In [6]:
idx_2_class = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
class_2_idx = {value: key for key, value in idx_2_class.items()}


In [7]:
train_data["label"] = train_data["label"].replace(class_2_idx)
test_data["label"] = test_data["label"].replace(class_2_idx)
val_data["label"] = val_data["label"].replace(class_2_idx)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14948\827028267.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data["label"] = train_data["label"].replace(class_2_idx)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14948\827028267.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data["label"] = test_data["label"].replace(class_2_idx)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14948\827028267.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old beh

In [8]:
train_data

,text,label
0,"I didn't feel humiliated, which was a surprise...",0
2,I can go from feeling so hopeless to so damned...,0
3,I'm grabbing a minute to post because I feel i...,3
4,I am ever feeling nostalgic about the fireplac...,2
5,"I am feeling extremely grouchy today, and to b...",3
...,...,...
395,I feel incredibly thankful for the lessons I'm...,1
396,I feel such a profound and unshakeable longing...,2
397,I feel distinctly called in Clermont to focus ...,2
398,I hope you can feel glad that she gave you so ...,1


In [9]:
from datasets import Dataset
ds_train = Dataset.from_pandas(train_data)
ds_test = Dataset.from_pandas(test_data)
ds_val = Dataset.from_pandas(val_data)

In [10]:
ds_train = ds_train.remove_columns(['__index_level_0__'])

In [11]:
ds_val

Dataset({
    features: ['text', 'label'],
    num_rows: 200
})

In [12]:
from transformers import (
     AutoTokenizer,
    AutoModelForSequenceClassification ,
)
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id,cache_dir="../tokenizer_cache")
device

'cuda'

In [13]:
max_len = 0 
def find_max_len(paragraph):
    global max_len
    len_sentence = len(paragraph)
    max_len = max(max_len, len_sentence)

In [14]:
max_input_length =512 # 27814
max_output_length = 8
batch_size = 2

In [15]:
max_len

0

In [16]:
def process_data_to_model_inputs(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length= max_input_length)

In [17]:
train_dataset = ds_train.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
)
val_dataset = ds_val.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
)

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
test_dataset = ds_test.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"],
)

test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"],
)


In [20]:
from datasets import Value 
train_dataset = train_dataset.cast_column("label", Value("int64"))
val_dataset = val_dataset.cast_column("label", Value("int64"))
test_dataset = test_dataset.cast_column("label", Value("int64"))

Casting the dataset:   0%|          | 0/399 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [21]:
num_labels = len(idx_2_class)
num_labels

6

In [22]:
from transformers import BitsAndBytesConfig
from peft import prepare_model_for_kbit_training,LoraConfig,PeftModel,get_peft_model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16")

In [23]:
robert_model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    cache_dir="../xlm_robert_model",
    # quantization_config=bnb_config,          
    id2label=idx_2_class,
    label2id=class_2_idx,
    num_labels=num_labels  
).to(device)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "query",
        "key",
        "value",
        "out_proj",
    ],
)
roberta_model = prepare_model_for_kbit_training(robert_model)
model = get_peft_model(roberta_model, config).to(device)

In [25]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 399
})

In [26]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        "eval_loss": eval_pred.loss if hasattr(eval_pred, "loss") else None  # Optional: include loss
    }

In [38]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="../xlm_robert_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
)

In [39]:
trainer = Trainer(
    model=robert_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [40]:
# Ensure label column is int for all datasets
trainer.train()

f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.985000,1.490089,0.415000,0.302919
2,1.301600,1.122050,0.685000,0.604875
3,0.715500,1.031004,0.725000,0.675275
4,1.025200,0.993572,0.695000,0.631155
5,1.034300,1.018577,0.720000,0.654797
6,0.481600,1.009667,0.765000,0.712014
7,0.145200,0.901203,0.785000,0.730662
8,1.627700,0.835202,0.760000,0.710124
9,0.570400,0.780847,0.785000,0.740301
10,0.432900,0.828768,0.785000,0.741486


f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
f:\pytho

TrainOutput(global_step=2000, training_loss=1.0042319114804268, metrics={'train_runtime': 641.4638, 'train_samples_per_second': 6.22, 'train_steps_per_second': 3.118, 'total_flos': 1055348939796480.0, 'train_loss': 1.0042319114804268, 'epoch': 10.0})

In [41]:
trainer.evaluate()

f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.7808469533920288,
 'eval_accuracy': 0.785,
 'eval_f1': 0.7403013171815489,
 'eval_runtime': 14.8361,
 'eval_samples_per_second': 13.481,
 'eval_steps_per_second': 6.74,
 'epoch': 10.0}

In [42]:
results = trainer.predict(test_dataset)

f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [43]:
logits = results.predictions
labels = results.label_ids
metrics = compute_metrics((logits, labels))
print(metrics)

{'accuracy': 0.81, 'f1': 0.7796394686907021, 'eval_loss': None}


In [44]:
SAVE_PATH = os.path.join(".." , "text_save_model", "xlm_robert_model_1")
os.makedirs(SAVE_PATH, exist_ok=True)
trainer.save_model(SAVE_PATH)